In [1]:
import sys
sys.path.append(r'..')
from data import read_svm_data
from cvxopt import matrix, solvers, spmatrix
import numpy as np


In [2]:
def standardize(X):
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_std[X_std == 0] = 1
    X_standardized = (X - X_mean) / X_std
    return X_standardized

def cov(X):
    m = X.shape[0]
    covariance_matrix = np.dot(X.T, X) / m
    return covariance_matrix

def get_eigenvectors(X, top_n):
    X_standardized = standardize(X)
    covariance_matrix = cov(X_standardized)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    sorted_idx = np.argsort(eigenvalues)[::-1]
    selected_eigenvectors = eigenvectors[:, sorted_idx[:top_n]]
    return selected_eigenvectors, np.mean(X, axis=0), np.std(X, axis=0)

def pca(X, components, mean, std):
    std[std == 0] = 1
    X_standardized = (X - mean) / std
    return np.dot(X_standardized, components)

In [3]:
training_labels, training_images = read_svm_data("training", r"../../MNIST_ORG", [2, 3, 8, 9])
testing_labels, testing_images = read_svm_data("testing", r"../../MNIST_ORG", [2, 3, 8, 9])

training_images.shape, training_labels.shape, testing_images.shape, testing_labels.shape

((20000, 784), (20000,), (3974, 784), (3974,))

In [4]:
components, train_mean, train_std = get_eigenvectors(training_images, 10)
training_images = pca(training_images, components, train_mean, train_std)
testing_images = pca(testing_images, components, train_mean, train_std)

In [5]:
C = 1
N = training_labels.shape[0]  # number of training samples
d = training_images.shape[1]  # dimension of each sample
labels_to_classify = [2, 3, 8, 9]
classifiers = {}

Part A

In [6]:
N = training_labels.shape[0]
d = training_images.shape[1]

Q_rows = []
Q_cols = []
Q_vals = []

# identity for w part
for i in range(d):
    Q_rows.append(i)
    Q_cols.append(i)
    Q_vals.append(1.0)

# ensuring that the slack variables' matrix is positive semi-definite (otherwise cvxopt raises an error)
for i in range(N):
    Q_rows.append(d + 1 + i)
    Q_cols.append(d + 1 + i)
    Q_vals.append(1e-6)

# sparse Q matrix
Q = spmatrix(Q_vals, Q_rows, Q_cols, (d + N + 1, d + N + 1), 'd')

# p vector
p = matrix([0.0] * (d + 1) + [C] * N)


solvers.options['maxiters'] = 30

for label in labels_to_classify:
    print(f"Training classifier for digit {label}...")
    yn = np.where(training_labels == label, 1, -1)
    
    A_rows, A_cols, A_vals = [], [], []

    # constructing sparse A matrix for the constraint y_i(w*x_i​ + b) >= 1 − e_i
    # converted to −y_i(w*x_i + b) − e_i <= −1, which is Gx <= h form
    for i in range(N):
        # -y_i * x_i
        for j in range(d):
            if training_images[i, j] != 0:
                A_rows.append(i)
                A_cols.append(j)
                A_vals.append(float(-yn[i] * training_images[i, j]))
        # -y_i
        A_rows.append(i)
        A_cols.append(d)
        A_vals.append(float(-yn[i]))
        
        # slack variable
        A_rows.append(i)
        A_cols.append(d + 1 + i)
        A_vals.append(-1.0)

    # constraint e_i >= 0
    for i in range(N):
        A_rows.append(N + i)
        A_cols.append(d + 1 + i)
        A_vals.append(-1.0)

    A = spmatrix(A_vals, A_rows, A_cols, (N + N, d + N + 1))

    # c vector
    c = matrix([-1.0] * N + [0.0] * N)
    
    # solve
    sol = solvers.qp(Q, p, A, c)

    w = np.array(sol['x'][:d]).flatten()
    b = sol['x'][d]

    classifiers[label] = (w, b)
    
def predict(X, classifiers):
    predictions = {label: np.dot(X, w) + b for label, (w, b) in classifiers.items()}
    final_predictions = np.fromiter((max(predictions, key=lambda x: predictions[x][i]) for i in range(len(X))), dtype=int)
    return final_predictions

predictions = predict(training_images, classifiers)
mapped_labels = np.array([label if label in labels_to_classify else None for label in training_labels])
correct_predictions = np.sum(predictions == mapped_labels)
accuracy = correct_predictions / len(training_labels)
print(f"Part A Training Accuracy: {accuracy * 100:.2f}%")

predictions = predict(testing_images, classifiers)
mapped_labels = np.array([label if label in labels_to_classify else None for label in testing_labels])
correct_predictions = np.sum(predictions == mapped_labels)
accuracy = correct_predictions / len(testing_labels)
print(f"Part A Testing Accuracy: {accuracy * 100:.2f}%")


Training classifier for digit 2...
     pcost       dcost       gap    pres   dres
 0: -1.4873e+04  6.8384e+04  5e+05  6e+00  2e+03
 1:  3.7203e+04 -4.1980e+04  1e+05  8e-01  2e+02
 2:  1.5461e+04 -5.6921e+03  2e+04  2e-01  5e+01
 3:  6.2069e+03  4.7036e+02  6e+03  4e-02  1e+01
 4:  3.8406e+03  1.9708e+03  2e+03  1e-02  3e+00
 5:  3.5858e+03  2.1552e+03  2e+03  6e-03  2e+00
 6:  3.3347e+03  2.3065e+03  1e+03  4e-03  1e+00
 7:  3.1702e+03  2.4033e+03  8e+02  3e-03  8e-01
 8:  3.0601e+03  2.4665e+03  6e+02  2e-03  6e-01
 9:  2.9862e+03  2.5089e+03  5e+02  1e-03  4e-01
10:  2.9008e+03  2.5564e+03  4e+02  8e-04  2e-01
11:  2.8605e+03  2.5773e+03  3e+02  6e-04  2e-01
12:  2.8152e+03  2.6039e+03  2e+02  4e-04  1e-01
13:  2.7810e+03  2.6238e+03  2e+02  2e-04  7e-02
14:  2.7604e+03  2.6364e+03  1e+02  2e-04  5e-02
15:  2.7416e+03  2.6475e+03  1e+02  9e-05  3e-02
16:  2.7227e+03  2.6604e+03  6e+01  5e-05  1e-02
17:  2.7136e+03  2.6671e+03  5e+01  3e-05  9e-03
18:  2.7055e+03  2.6730e+03  3e+01 

Part B

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

for class_label in [2,3,8,9]:
    print(f"\nTraining classifier for class {class_label}")
    # 1 for the current class, 0 for all others
    binary_target = (training_labels == class_label).astype(int)
    
    classifier = SVC(kernel='linear', C=1, verbose=True)
    classifier.fit(training_images, binary_target)
    classifiers[class_label] = classifier
    
def predict_one_vs_rest(classifiers, images):
    scores = np.column_stack([
        clf.decision_function(images) for clf in classifiers.values()
    ])
    predicted_class_indices = np.argmax(scores, axis=1)
    predicted_classes = [[2,3,8,9][i] for i in predicted_class_indices]
    
    return predicted_classes

predictions = predict_one_vs_rest(classifiers, training_images)
accuracy = accuracy_score(training_labels, predictions)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

predictions = predict_one_vs_rest(classifiers, testing_images)
accuracy = accuracy_score(testing_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Training classifier for class 2
[LibSVM]
Training classifier for class 3
[LibSVM]
Training classifier for class 8
[LibSVM]
Training classifier for class 9
[LibSVM]Training Accuracy: 89.10%
Test Accuracy: 89.13%


Part C

In [8]:
def polynomial_kernel(X1, X2, degree=2):
    K = np.dot(X1, X2.T)
    return (1 + K) ** degree

solvers.options['show_progress'] = False

N = training_images[::10].shape[0]


for label in labels_to_classify:
    print(f"\nTraining classifier for digit {label}...")
    yn = np.where(training_labels[::10] == label, 1, -1)

    print("Building Kernel matrix...")
    
    # Q is a NxN matrix, where N is the number of training samples
    # Q[i, j] = y[i] * y[j] * K(x[i], x[j])
    K = polynomial_kernel(training_images[::10], training_images[::10])
    Y = yn.reshape(-1, 1) * yn.reshape(1, -1)
    Q = matrix(K * Y, tc='d')

    # p is a Nx1 matrix of -1s
    p = matrix(-np.ones(N))
    
    # Ax = c equality constraint sum(alpha * y) = 0
    # A is a row vector of labels y
    # c is a scalar 0
    
    A = matrix(yn, (1, N), 'd')
    c = matrix(0.0)
    
    # Gx <= h inequality constraint 0 <= alpha <= C
    # lower bound 0 <= alpha
    G = spmatrix([], [], [], (N, N), 'd')
    G[::N+1] = -1
    h = matrix(np.zeros(N), tc='d')
    
    # upper bound alpha <= C
    G_up = spmatrix([], [], [], (N, N), 'd')
    G_up[::N+1] = 1
    h_up = matrix(C * np.ones(N), tc='d')
    
    # concatenate lower and upper bound
    G = matrix([G, G_up])
    h = matrix([h, h_up])

    print("Solving QP...")
    solution = solvers.qp(Q, p, G, h, A, c)
    alphas = np.array(solution['x']).flatten()

    # support vectors have non zero lagrange multipliers
    # alphas > 0 caused numerical problems, so we use 1e-6
    sv = alphas > 1e-6
    ind = np.arange(len(alphas))[sv]    # indices of support vectors
    alpha_sv = alphas[sv]               # alphas of support vectors
    sv_y = yn[sv]                       # labels of support vectors
    sv_X = training_images[::10][sv]      # support vectors

    # b = 1/N * sum(y - sum(alpha * y * K))
    b = np.mean(sv_y - np.sum(alpha_sv * sv_y * K[np.ix_(ind, sv)], axis=0))

    classifiers[label] = (alpha_sv, sv_X, sv_y, b)
    
def predict(X, classifiers):
    results = {}
    for label, (alphas, support_vectors, sv_labels, b) in classifiers.items():
        K_eval = polynomial_kernel(X, support_vectors)
        prediction = np.dot(K_eval, alphas * sv_labels) + b
        results[label] = prediction
    predictions = np.argmax(np.column_stack([results[label] for label in labels_to_classify]), axis=1)
    mapped_labels = [labels_to_classify[i] for i in predictions]
    return np.array(mapped_labels)

# Prediction and accuracy calculation
predictions = predict(training_images, classifiers)
accuracy = np.mean(predictions == training_labels)
print(f"Part C Training Accuracy: {accuracy * 100:.2f}%")

predictions = predict(testing_images, classifiers)
accuracy = np.mean(predictions == testing_labels)
print(f"Part C Testing Accuracy: {accuracy * 100:.2f}%")


Training classifier for digit 2...
Building Kernel matrix...
Solving QP...

Training classifier for digit 3...
Building Kernel matrix...
Solving QP...

Training classifier for digit 8...
Building Kernel matrix...
Solving QP...

Training classifier for digit 9...
Building Kernel matrix...
Solving QP...
Part C Training Accuracy: 93.11%
Part C Testing Accuracy: 92.98%


Part D

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

classifiers = {}
classes = [2, 3, 8, 9]
for class_label in classes:
    print(f"\nTraining classifier for class {class_label}")
    # 1 for the current class, 0 for all others
    binary_target = (training_labels == class_label).astype(int)
    
    classifier = SVC(kernel='poly', C=1, verbose=True)
    classifier.fit(training_images, binary_target)
    classifiers[class_label] = classifier

def predict_one_vs_rest(classifiers, images):
    scores = np.column_stack([
        clf.decision_function(images) for clf in classifiers.values()
    ])
    predicted_class_indices = np.argmax(scores, axis=1)
    predicted_classes = [classes[i] for i in predicted_class_indices]
    
    return predicted_classes

predictions = predict_one_vs_rest(classifiers, training_images)
accuracy = accuracy_score(training_labels, predictions)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

predictions = predict_one_vs_rest(classifiers, testing_images)
accuracy = accuracy_score(testing_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Training classifier for class 2
[LibSVM]
Training classifier for class 3
[LibSVM]
Training classifier for class 8
[LibSVM]
Training classifier for class 9
[LibSVM]Training Accuracy: 93.64%
Test Accuracy: 93.28%
